In [9]:
import json
from openai import OpenAI
from dataclasses import dataclass, field
from typing import List, Union
from pprint import pprint

In [14]:
# Setup OpenAI client to use Ollama
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

@dataclass
class Agent:
    """Agent class similar to agent.Agent with handoffs"""
    name: str
    instructions: str
    handoffs: List['Agent'] = field(default_factory=list)
    model: str = "qwen3:8b"

class Runner:
    """Runner class similar to agent.Runner with handoff support"""
    
    @classmethod
    def run_sync(cls, agent: Agent, input: str, max_turns: int = 10) -> str:
        """Run agent with handoff support"""
        messages = [
            {"role": "system", "content": agent.instructions},
            {"role": "user", "content": input}
        ]
        
        current_agent = agent
        
        for turn in range(max_turns):
            print(f"Turn {turn + 1} - Agent: {current_agent.name}")
            
            # Prepare handoff tools
            handoff_tools = []
            for handoff_agent in current_agent.handoffs:
                handoff_tools.append({
                    "type": "function",
                    "function": {
                        "name": f"handoff_to_{handoff_agent.name.lower().replace(' ', '_')}",
                        "description": f"Transfer conversation to {handoff_agent.name}",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "reason": {
                                    "type": "string",
                                    "description": "Reason for handoff"
                                }
                            },
                            "required": ["reason"]
                        }
                    }
                })
            print("-"*30)
            
            pprint(messages)
            print("-"*30)
            pprint(handoff_tools)
            print("+"*30)
            
            response = client.chat.completions.create(
                model=current_agent.model,
                messages=messages,
                tools=handoff_tools if handoff_tools else None
            )
            pprint(response.model_dump())
            print("++"*40)
            message = response.choices[0].message
            
            if message.tool_calls:
                # Check for handoffs
                for tool_call in message.tool_calls:
                    function_name = tool_call.function.name
                    
                    if function_name.startswith("handoff_to_"):
                        # Extract target agent name
                        target_name = function_name.replace("handoff_to_", "").replace("_", " ").title()
                        
                        # Find target agent
                        target_agent = None
                        for handoff_agent in current_agent.handoffs:
                            if handoff_agent.name.lower().replace(" ", "_") == target_name.lower().replace(" ", "_"):
                                target_agent = handoff_agent
                                break
                        
                        if target_agent:
                            arguments = json.loads(tool_call.function.arguments)
                            reason = arguments.get("reason", "No reason provided")
                            
                            print(f"  🔄 Handoff: {current_agent.name} → {target_agent.name}")
                            print(f"  Reason: {reason}")
                            
                            # Switch to new agent
                            current_agent = target_agent
                            
                            # Update system message for new agent
                            messages[0] = {"role": "system", "content": current_agent.instructions}
                            
                            # Add handoff confirmation
                            messages.append({
                                "role": "tool",
                                "content": f"Transferred to {target_agent.name}",
                                "tool_call_id": tool_call.id
                            })
                            break
            else:
                # Final response
                print(f"  Final: {message.content}")
                return message.content
        
        return "Max turns reached"


In [15]:
# Create specialized agents
weather_agent = Agent(
    name="Weather Agent",
    instructions="You are a weather specialist. Only answer weather-related questions. Be concise."
)

math_agent = Agent(
    name="Math Agent",
    instructions="You are a math specialist. Only solve mathematical problems. Show your work briefly."
)

research_agent = Agent(
    name="Research Agent", 
    instructions="You are a research specialist. Only answer research and information questions."
)

# Create triage agent with handoffs (like agent.Agent with handoffs)
triage_agent = Agent(
    name="Triage Agent",
    instructions="""You are a triage agent that routes requests to specialists.
    - For weather questions, handoff to Weather Agent
    - For math questions, handoff to Math Agent  
    - For research questions, handoff to Research Agent
    - For simple greetings, answer directly""",
    handoffs=[weather_agent, math_agent, research_agent]
)

# Test cases
Runner.run_sync(triage_agent, "Calculate 15 * 23 + 7")

Turn 1 - Agent: Triage Agent
------------------------------
[{'content': 'You are a triage agent that routes requests to specialists.\n'
             '    - For weather questions, handoff to Weather Agent\n'
             '    - For math questions, handoff to Math Agent  \n'
             '    - For research questions, handoff to Research Agent\n'
             '    - For simple greetings, answer directly',
  'role': 'system'},
 {'content': 'Calculate 15 * 23 + 7', 'role': 'user'}]
------------------------------
[{'function': {'description': 'Transfer conversation to Weather Agent',
               'name': 'handoff_to_weather_agent',
               'parameters': {'properties': {'reason': {'description': 'Reason '
                                                                       'for '
                                                                       'handoff',
                                                        'type': 'string'}},
                              'required': ['r

"<think>\nOkay, let's see. The problem is to calculate 15 multiplied by 23 and then add 7. Hmm, right. So, first step is to do the multiplication part. Let me recall how to multiply 15 by 23. \n\nMaybe I can break it down. 15 times 20 is straightforward. 15*20 is 300. Then 15 times 3 is 45. So adding those together, 300 + 45 equals 345. Wait, is that right? Let me check another way. Alternatively, I can use the standard multiplication method. \n\nSo, 23 multiplied by 15. Let me write it out:\n\n   23\nx 15\n-------\nFirst, multiply 23 by 5. That's 115. Then multiply 23 by 10, which is 230. Then add 115 and 230. 115 + 230 is 345. Yep, that matches. So 15*23 is indeed 345. \n\nNow, the next step is to add 7 to that result. So 345 + 7. That should be 352. Wait, 345 plus 7... 345 + 5 is 350, then +2 more is 352. Yeah, that seems right. \n\nLet me double-check the entire calculation to be sure. Maybe using another method. Let's think about 15*23. If I consider 15 as 10 + 5, then 10*23 is 23